Explore the DataSet in this Notebook

In [1]:
import pandas as pd
import os
import glob
from groq import Groq
# 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

In [2]:
r2r_dataset = pd.read_json("Dataset/R2R_train_updated.json")
original_dataset = r2r_dataset.copy()
filtered_dataset = r2r_dataset[r2r_dataset['scan'] == "5LpN3gDmAk7"]

In [3]:
import json

client = Groq(
    api_key="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",
)

output_dicts = []
count = 0
for i, row in filtered_dataset.iterrows():
    instructions_json = json.dumps(row['instructions'], indent=2)
    if(count%10 == 0):
        print("i = ",count)
    count+=1
    # print("Instructions = ", instructions_json)
    try:
        chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": """You are a navigation goal statement generator given a list of instructions, generate goal statement without the instructions.that responds in JSON schema which should include a list of dictionaries, where each dictionary has keys 'instruction_index' and 'goal statement'. Generate a list of dictionaries like this example."},:
        {
"navigation_goals": [
    {
        "instruction_index": 1,
        "goal_statement": "I want to navigate to the Hallway past the kitchen."
    },
    {
        "instruction_index": 2,
        "goal_statement": "I want to navigate to the Hallway past the clear glass table."
    },
    {
        "instruction_index": 3,
        "goal_statement": "I want to navigate to the Hallway past the Counter on the right."
    }
]
} """},
            {
                "role": "user",
                "content": f"""
            Given the following instructions ,generate a precise goal statement for each instruction, that the robot and generate instructions on his own.Give your answer in a single dictionary, which contains a list dictionaries.Describe the current scene and where to navigate to. The instructions are:
            {instructions_json}
            Note: Donot give me instructions,just a goal statement telling what the above instructions want to acheive, e.g I am here and want to go to there.""",
            }
        ],
        response_format={"type": "json_object"},
        model="llama3-8b-8192",
    )
        # print(chat_completion.choices[0].message.content)
        output_dicts.append(chat_completion.choices[0].message.content)
    except:
        chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": """You are a navigation goal statement generator given a list of instructions, generate goal statement without the instructions.that responds in JSON schema which should include a list of dictionaries, where each dictionary has keys 'instruction_index' and 'goal statement'. Generate a list of dictionaries like this example."},:
        {
"navigation_goals": [
    {
        "instruction_index": 1,
        "goal_statement": "I want to navigate to the Hallway past the kitchen."
    },
    {
        "instruction_index": 2,
        "goal_statement": "I want to navigate to the Hallway past the clear glass table."
    },
    {
        "instruction_index": 3,
        "goal_statement": "I want to navigate to the Hallway past the Counter on the right."
    }
]
} """},
            {
                "role": "user",
                "content": f"""
            Given the following instructions ,generate a precise goal statement for each instruction, that the robot and generate instructions on his own.Give your answer in a single dictionary, which contains a list dictionaries.Describe the current scene and where to navigate to. The instructions are:
            {instructions_json}
            Note: Donot give me instructions,just a goal statement telling what the above instructions want to acheive, e.g I am here and want to go to there.""",
            }
        ],
        response_format={"type": "json_object"},
        model="llama3-8b-8192",
    )
        # print(chat_completion.choices[0].message.content)
        output_dicts.append(chat_completion.choices[0].message.content)

i =  0
i =  10
i =  20
i =  30
i =  40
i =  50
i =  60
i =  70
i =  80


In [4]:
goalList = []
for dic in output_dicts:
    goalList.append([inst['goal_statement'] for inst in json.loads(dic)['navigation_goals']])

# filtered_dataset['goals'] = None
j = 0
for i, row in filtered_dataset.iterrows():
    filtered_dataset.at[i, 'goals'] = goalList[j]
    j+=1
    

In [5]:
# original_dataset['goals'] = None
for i, row in filtered_dataset.iterrows():
    # Find the corresponding row in the original dataset
    original_index = original_dataset.index[original_dataset['path_id'] == row['path_id']].tolist()
    if original_index:
        original_dataset.at[original_index[0], 'goals'] = row['goals']

# Save the updated full dataset to a new JSON file
updated_full_dataset_path = "Dataset/R2R_train_updated.json"
original_dataset.to_json(updated_full_dataset_path, orient="records", lines=True)

print(f"Updated dataset saved to {updated_full_dataset_path}")

Updated dataset saved to Dataset/R2R_train_updated.json


In [7]:
path = r2r_dataset.iloc[0]['path']

print(path)
# Specify the directory where your images are stored
folder_path = 'Dataset/17DRP5sb8fy/matterport_color_images'

# Specify the word the image names should start with
start_word = path[0]

# Construct the search pattern
pattern = os.path.join(folder_path, f'{start_word}*')

# Get a list of all image filenames that start with the specific word
image_files = glob.glob(pattern)

# Print the filenames (only the names, not the full paths)
image_names = [os.path.basename(image) for image in image_files]
print(image_names[5:8])

['10c252c90fa24ef3b698c6f54d984c5c', '77a1a11978b04e9cbf74914c98578ab8', 'b185432bf33645aca813ac2a961b4140', '5e9f4f8654574e699480e90ecdd150c8', '08c774f20c984008882da2b8547850eb', 'da5fa65c13e643719a20cbb818c9a85d']
['10c252c90fa24ef3b698c6f54d984c5c_i0_5.jpg', '10c252c90fa24ef3b698c6f54d984c5c_i1_0.jpg', '10c252c90fa24ef3b698c6f54d984c5c_i1_1.jpg']


# Generated Goals and Dataset Intrsuctions Similarity Check

In [12]:
!pip3 install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
import pandas as pd
import json
from sentence_transformers import SentenceTransformer, util  # For embedding and similarity

# Initialize the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose any other suitable model


# Similarity function
def is_similar(embedding1, embedding2, threshold=0.55):
    similarity = util.pytorch_cos_sim(embedding1, embedding2)
    if(similarity.item() < threshold):
        print("Sim = ",similarity.item())
    return similarity.item() > threshold

# Only checking for first training case
r2r_dataset = pd.read_json("Dataset/R2R_train_updated.json")
r2r_dataset = r2r_dataset[r2r_dataset['scan'] == "17DRP5sb8fy"]
# img_folder_path = 'Dataset/17DRP5sb8fy/matterport_color_images'

total = 0
total_correct = 0
total_incorrect = 0
for index, row in r2r_dataset.iterrows():
    instructions = row['instructions']
    goals = row['goals']
    
    for i in range(len(goals)):
        total += 1
        goal_embedding = embedding_model.encode(goals[i])
        instruction_embedding = embedding_model.encode(instructions[i])
        if not is_similar(goal_embedding, instruction_embedding):
            total_incorrect += 1
            print("Goal = ",goals[i])
            print("Instruction = ",instructions[i])
            print("Not similar for path = ",row['path_id'])
        else:
            total_correct += 1

print("Total = ",total)
print("Total Correct = ",total_correct)
print("Total Incorrect = ",total_incorrect)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Sim =  0.46148812770843506
Goal =  I am in the sitting room and I want to navigate to right after the open door.
Instruction =  Walk through the sitting room past the picture of the mermaid.  Stop right after the open door. 
Not similar for path =  6380
Sim =  0.22321462631225586
Goal =  I am here, want to go to the area just past the corner.
Instruction =  Move forward and turn left. Your goal is ahead, move forward until you reach it. 
Not similar for path =  790
Sim =  0.5227680206298828
Goal =  I am here and want to navigate to the glass table past the furniture arrangement.
Instruction =  Go to the lamp. Go to the painting. Go to the round table. Go to the blue pillow. Go to the glass table. 
Not similar for path =  3070
Total =  90
Total Correct =  87
Total Incorrect =  3
